In [34]:
# !pip install lightgbm

In [123]:
import numpy as np
import pandas as pd

from lightgbm import LGBMClassifier
from sklearn.base import BaseEstimator, ClassifierMixin

from metrics import dice_coefficient

In [124]:
metrics_dict = {
    "dice_coefficient": {"2d": dice_coefficient, "3d": dice_coefficient}
}

unary_metrics_dict = {
    "area": {"2d": lambda x: (x > 0).sum(), "3d": lambda x: (x > 0).sum()}
}

In [125]:
class BaseQualityEstimator(BaseEstimator, ClassifierMixin):
    """Base Estimator for segmentation quality assessment"""

    def __init__(self, metrics=["dice_coefficient"], unary_metrics=["area"], meta_clf=LGBMClassifier()):
        """
        Args:
            metrics: list of strings: metrics to be computed on pairs of preds and gt
            unary_metrics: list of string: metrics to be computed on preds directly
        
        TODO: params??
        """
        self.meta_clf = meta_clf
        self.metrics = list(filter(lambda _: _ in metrics_dict, metrics))
        self.unary_metrics = list(filter(lambda _: _ in unary_metrics_dict, unary_metrics))
        
        self.data_type = None
        self.X_metrics = None

    def fit(self, X, Xy=None, y=None):
        """
        
        """
        assert len(X) == len(Xy) == len(y)
        # get the dimensionality of the data
        self.data_type = self._check_data_type(X)
        # compute all the metrics on the pairs from X (predictions) and Xy (gt)
        self.X_metrics = self._compute_metrics(X, Xy)
        # fit meta-classifier to metrics and human-made labels
        self.meta_clf.fit(self.X_metrics, y)

        return self
    
    def predict(self, X, Xy):
        
        y_prob = self.meta_clf.predict_proba(X)
        
        return (y_prob > 0.5).astype(float)
    
    def predict_proba(self, X, Xy):
        
        X_metrics = self._compute_metrics(X, Xy)
        
        y_pred = self.meta_clf.predict_proba(X_metrics)
        
        return y_pred
    
    def _compute_metrics(self, X, Xy):
        
        def _metrics(x, xy):
            metrics_computed = dict()
            for metric_ in self.metrics:
                metrics_computed[metric_] = metrics_dict[metric_][self.data_type](x, xy)
            return metrics_computed
        
        def _unary_metrics(x):
            unary_metrics_computed = dict()
            for metric_ in self.unary_metrics:
                unary_metrics_computed[metric_] = unary_metrics_dict[metric_][self.data_type](x)
            
            return unary_metrics_computed
        
        metrics_computed = []
        
        for x_, xy_ in zip(X, Xy):
            metrics_temp_ = _metrics(x_, xy_)
            metrics_temp_.update(_unary_metrics(x_))
            metrics_computed.append(metrics_temp_)
            
        df_metrics_computed = pd.DataFrame(metrics_computed)
        
        return df_metrics_computed
        
    def _check_data_type(self, X):
        """
        TODO:
        """
        # заглушка:
        if len(X.shape) == 2:
            return "2d"
        elif X.shape[2] == 1:
            return "2d"
        else:
            return "3d"

    def score(self, X, y=None):
        # counts number of values bigger than mean
        return(sum(self.predict(X))) 

### Check metrics:

In [126]:
X = np.random.randint(0, 1 + 1, size=[10, 300, 300, 3])
Xy = np.random.randint(0, 1 + 1, size=[10, 300, 300, 3])
y = np.random.randint(0, 1 + 1, size=[10])

In [127]:
dice_coefficient(img_1, img_2)

0.5000644068517781

In [128]:
q_clf = BaseQualityEstimator( metrics=["dice_coefficient"], unary_metrics=["area"], meta_clf=LGBMClassifier())

In [129]:
q_clf.fit(X, Xy, y)

BaseQualityEstimator()

In [130]:
q_clf.predict_proba(X, Xy)

array([[0.6, 0.4],
       [0.6, 0.4],
       [0.6, 0.4],
       [0.6, 0.4],
       [0.6, 0.4],
       [0.6, 0.4],
       [0.6, 0.4],
       [0.6, 0.4],
       [0.6, 0.4],
       [0.6, 0.4]])

In [87]:
d = dict({1: 2})
c = dict({2: 3})

In [88]:
d.update(c)

In [90]:
d.

TypeError: unsupported operand type(s) for +: 'dict' and 'dict'